In [1]:
%load_ext autoreload
%autoreload 2

from members import *
from food import *

['Авраменко', 'Бубнова', 'Виноградова', 'Герасимов', 'Дятлов', 'Еналеев', 'Лебедев', 'Малышев', 'Ткачева']
Количество участников: 9
Количество женщин: 3


# Рассчет

In [2]:
NBreakfast = 6
NDinner = 6
NLunch = 6
NSnack = 6

WomanFoodMaxWeight = 2000

Food = [# Продукты "ежедневного" потребления
         ('Соль'                      ,    7,    6,       Дятлов),
         ('Сахар'                     ,   60,    6,       ([Еналеев, Дятлов], [3.2, 2.8])),
         ('Чай (зеленый / черный)'    ,    7,    6,       Ткачева),
         ('Лимоны в сахаре'           ,   20,    6,       Дятлов),
         ('Перец'                     ,    1,    6,       Дятлов),
         ('Хмели-сунели'              ,    1,    6,       Дятлов),
         ('Аджика'                    ,    3,    6,       Ткачева),

        # Завтрак
        
         ('Геркулес Традиционный (завтрак)' ,   40,    2,       Герасимов),
         ('Рис (завтрак)'             ,   50,    2,       Герасимов),
         ('Гречка (завтрак)'          ,   50,    2,       Герасимов),
         ('Сухое молоко (завтрак)'    ,   25,    6,       Авраменко),
         ('Колбаса (завтрак)'         ,   40,    3,       Герасимов),
         ('Сыр (завтрак)'             ,   40,    3,       Герасимов), 
         ('Изюм (завтрак)'            ,   5,     6,       Малышев),
         ('Чернослив (завтрак)'       ,   20,    3,       Ткачева),
         ('Cухое молоко в кофе (завтрак)'   ,   2,     6,       Авраменко),
         ('Кофе'                      ,   2,     6,       Ткачева),
     
        # Перекус
     
         ('Колбаса (перекус)'         ,   40,    3,       Авраменко),
         ('Сыр (перекус)'             ,   40,    3,       Авраменко),
         ('Шоколад (перекус)'         ,   25,    6,       Виноградова),
         ('Орехи (перекус)'           ,   20,    6,       Авраменко),
         ('Сухари (перекус)'          ,   40,    6,       ([Виноградова, Малышев, Авраменко], [2, 2, 2])), 
         ('Курага (перекус)'          ,   10,    6,       Ткачева),
         ('Чернослив (перекус)'       ,   10,    6,       Ткачева),
     
         # Обед
         
         ('Чеснок'                    ,    4,    6,       Герасимов),
         ('Супы'                      ,    35,   6,       Лебедев),
     
         # Ужин
     
         ('Рис (ужин)'                ,   80,    1,       Малышев),
         ('Гречка (ужин)'             ,   80,    2,       Малышев),
         ('Чечевица (ужин)'           ,   80,    1,       Малышев),
         ('Спагетти (ужин)'           ,   90,    2,       Малышев),
         ('Пеммикан (ужин)'           ,   81,    6,       Лебедев),
         ('Лук'                       ,   10,    6,       Герасимов),
         ('Сладости (ужин)'           ,   40,    6,       Бубнова),
         ('Спирт (ужин)'              ,   15,    6,       Еналеев),
        
        # Бензин
        
         ('Бензин'                    ,   100,   6,       ([Еналеев, Дятлов], [3.4, 2.6]))
]

table = food_planning(Food, 
                      NBreakfast=NBreakfast, 
                      NDinner=NDinner, 
                      NLunch=NLunch, 
                      NSnack=NSnack, 
                      WomanFoodMaxWeight=WomanFoodMaxWeight)

table

Общий вес: 40122.0
Средний вес на мужчину: 5687.0
Средний вес на женщину: 2000
Вес на человека в день: 743.0

Авраменко: 269.0
Бубнова: -160.0
Виноградова: -70.0
Герасимов: 251.0
Дятлов: 269.0
Еналеев: 89.0
Лебедев: -577.0
Малышев: 197.0
Ткачева: -268.0

Остаток: 0.0


,name,portion,times,weight,Кто несет
0,Соль,63.0,6.0,378.0,Дятлов
1,Сахар,540.0,3.2,1728.0,Еналеев
2,Сахар,540.0,2.8,1512.0,Дятлов
3,Чай (зеленый / черный),63.0,6.0,378.0,Ткачева
4,Лимоны в сахаре,180.0,6.0,1080.0,Дятлов
5,Перец,9.0,6.0,54.0,Дятлов
6,Хмели-сунели,9.0,6.0,54.0,Дятлов
7,Аджика,27.0,6.0,162.0,Ткачева
8,Геркулес Традиционный (завтрак),360.0,2.0,720.0,Герасимов
9,Рис (завтрак),450.0,2.0,900.0,Герасимов


# Таблица поручений по закупке

In [3]:
group_by_members(table, fname='food.txt')

Участник: Авраменко, несет: 5418.0
                             name  portion  times  weight  Кто несет
11         Сухое молоко (завтрак)    225.0    6.0  1350.0  Авраменко
16  Cухое молоко в кофе (завтрак)     18.0    6.0   108.0  Авраменко
18              Колбаса (перекус)    360.0    3.0  1080.0  Авраменко
19                  Сыр (перекус)    360.0    3.0  1080.0  Авраменко
21                Орехи (перекус)    180.0    6.0  1080.0  Авраменко
24               Сухари (перекус)    360.0    2.0   720.0  Авраменко



Участник: Бубнова, несет: 2160.0
               name  portion  times  weight Кто несет
35  Сладости (ужин)    360.0    6.0  2160.0   Бубнова



Участник: Виноградова, несет: 2070.0
                 name  portion  times  weight    Кто несет
20  Шоколад (перекус)    225.0    6.0  1350.0  Виноградова
22   Сухари (перекус)    360.0    2.0   720.0  Виноградова



Участник: Герасимов, несет: 5436.0
                               name  portion  times  weight  Кто несет
8   Геркулес

# Комментарии

### В 1.5л бутылке умещаятся 1150г бензина